In [4]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "bayessizer")))

In [9]:
import os
print(os.getcwd())  # Ensure this is the root of your project

/Users/b174270/bayes-sizer/bayessizer


In [2]:
import os
os.chdir("/Users/b174270/bayes-sizer")

FileNotFoundError: [Errno 2] No such file or directory: '~'

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import beta
from bayessizer.calculator import bayesian_sample_size
from bayessizer.inference import compute_posterior, credible_interval, probability_test_better
from bayessizer.priors import get_beta_prior


# Set notebook to display inline plots
%matplotlib inline

In [ ]:
from bayessizer.calculator import bayesian_sample_size

# Example: Calculate required sample size with default parameters
sample_size = bayesian_sample_size(prior_a=2, prior_b=2, min_effect=0.02, power=0.9, loss_threshold=0.005)
print(f"Required sample size per group for 2% effect size: {sample_size}")

In [14]:
# Example: Compute Bayesian sample size with different minimum effects
min_effects = np.linspace(0.01, 0.1, 10)
sample_sizes = [bayesian_sample_size(prior_a=2, prior_b=2, min_effect=e, power=0.9, loss_threshold=0.05) for e in min_effects]

# Plot the effect size vs. required sample size
plt.figure(figsize=(8, 5))
plt.plot(min_effects, sample_sizes, marker='o', linestyle='-')
plt.xlabel('Minimum Detectable Effect Size')
plt.ylabel('Required Sample Size per Group')
plt.title('Bayesian Sample Size Calculation')
plt.grid()
plt.show()

KeyboardInterrupt: 

In [ ]:
# Display a single sample size calculation
sample_size = bayesian_sample_size(prior_a=2, prior_b=2, min_effect=0.02, power=0.9, loss_threshold=0.01)
print(f"Required sample size per group for 2% effect size: {sample_size}")

# Plot posterior distributions for control and test groups
sample_n = 100  # Fixed sample size for visualization
control_posterior = beta(2 + sample_n * 0.5, 2 + sample_n * 0.5)
test_posterior = beta(2 + sample_n * (0.5 + 0.02), 2 + sample_n * (0.5 - 0.02))

x = np.linspace(0, 1, 1000)
control_pdf = control_posterior.pdf(x)
test_pdf = test_posterior.pdf(x)

plt.figure(figsize=(8, 5))
sns.lineplot(x=x, y=control_pdf, label='Control Group', color='blue')
sns.lineplot(x=x, y=test_pdf, label='Test Group', color='red')

# Compute 95% credible intervals
control_ci_lower, control_ci_upper = control_posterior.ppf([0.025, 0.975])
test_ci_lower, test_ci_upper = test_posterior.ppf([0.025, 0.975])

plt.axvline(control_ci_lower, color='blue', linestyle='--', alpha=0.7, label='Control 95% CI')
plt.axvline(control_ci_upper, color='blue', linestyle='--', alpha=0.7)
plt.axvline(test_ci_lower, color='red', linestyle='--', alpha=0.7, label='Test 95% CI')
plt.axvline(test_ci_upper, color='red', linestyle='--', alpha=0.7)

# Compute probability that the test group is better than the control group
prob_test_better = 1 - control_posterior.cdf(test_posterior.ppf(0.5))

plt.xlabel('Conversion Rate')
plt.ylabel('Density')
plt.title(f'Posterior Distributions with 95% Credible Intervals\nP(Test > Control) = {prob_test_better:.3f}')
plt.legend()
plt.grid()
plt.show()

print(f"Probability that the test group outperforms the control group: {prob_test_better:.3f}")

# Implement Bayesian stopping criteria
stopping_threshold = 0.95  # Stop when P(Test > Control) exceeds 95%

if prob_test_better >= stopping_threshold:
    print("Experiment can be stopped early: Test group is significantly better!")
else:
    print("Continue collecting more data: Decision is uncertain.")